In [17]:
import pandas as pd
import numpy as np
from utilsPtBr import Utils
import tensorflow as tf

dataPath = 'data/shortCorpus.csv'
df_uf = pd.read_csv(dataPath, sep=';')

util = Utils()

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [18]:
dictDataframeTreino = {
    "tag": [],
    "tokens": [],
    "types": [], 
    "freq": []
}

dictDataframeTeste = {
    "tag": [],
    "tokens": [],
    "types": [], 
    "freq": []
}

formatTag = {
    "Publicação - Despacho / Decisão": "decisao",
    "Publicação - Sentença": "sentenca",
    "Publicação - Audiência": "audiencia",
}


In [22]:
from random import randint
def calcTestLineAleatory():
    numberAleatory = randint(0,99);
    if numberAleatory % 9 == 1: 
        return True;
    return False;

In [23]:
# pre formatando texto e separando treino e teste
for i in range (len(df_uf.index)):
    tag = df_uf.loc[i].iloc[1]
    tag = formatTag.get(tag, "outros")
  
    text = df_uf.loc[i].iloc[2]
    tokens = util.tokenize(text)
    freq = util.getPropoFreq(tokens)
  
    if calcTestLineAleatory():
        dictDataframeTeste['tag'].append(tag)
        dictDataframeTeste['tokens'].append(tokens)
        dictDataframeTeste['types'].append(set(tokens))
        dictDataframeTeste['freq'].append(freq)
    else:
        dictDataframeTreino['tag'].append(tag)
        dictDataframeTreino['tokens'].append(tokens)
        dictDataframeTreino['types'].append(set(tokens))
        dictDataframeTreino['freq'].append(freq)
  
df_teste = pd.DataFrame(dictDataframeTeste)
df_treino = pd.DataFrame(dictDataframeTreino)

print(df_teste)
print(df_treino)

          tag                                             tokens  \
0     decisao  [código:90503231, código, relacional:1, data:6...   
1     decisao  [código:85908964, código, relacional:1, data:3...   
2    sentenca  [código:103077923, código, relacional:1, data:...   
3    sentenca  [código:92980142, código, relacional:1, data:2...   
4      outros  [código:107142594, código, relacional:0, data:...   
..        ...                                                ...   
599   decisao  [cã³digo, 375881872, texto, cã³digo, relaciona...   
600   decisao  [data, disponibilizacao, publicacao, fonte, of...   
601   decisao  [data, disponibilizacao, publicacao, fonte, of...   
602   decisao  [cã³digo, 376534571, texto, cã³digo, relaciona...   
603   decisao  [data, disponibilizacao, publicacao, fonte, of...   

                                                 types  \
0    {-n, relacional:1, cad, eduardo, 019.01.2008.0...   
1    {desconstituicao, contrato, 0630, comarca, san...   
2    {ass